In [1]:
import streamlit as st
import pandas as pd
import numpy as np
import pymysql
import matplotlib.pyplot as plt
import plotly.figure_factory as ff
import plotly.graph_objects as go
import plotly.express as px

def _call_db_info(db_table):
    return pymysql.connect(
        host = '10.94.77.9',
        port= 13306,
        user = 'dxbot1',
        password='ensol2020!',
        db = db_table,
        charset = 'utf8')

In [2]:
conn = _call_db_info("gscm_e61d")
read = conn.cursor()
tmp_rslt_sql = "select * from inven_day where ww=202101"
tmp_ref_sql = "select  ww, inven from inven_rslt where ww >= 202101"
tmp_lt_sql = "select * from lt_day"
read.execute(tmp_rslt_sql)    
inven_d = pd.DataFrame(read.fetchall())
read.execute(tmp_ref_sql)
inven = pd.DataFrame(read.fetchall())
read.execute(tmp_lt_sql)
lt_d = pd.DataFrame(read.fetchall())   
read.close()

inven.columns = ['ww', 'inven_true']
inven = inven.astype({'ww' : 'str'})
lt_d.columns = ['ww', 'lt_d']
lt_d = lt_d.astype({'ww' : 'str'})

tmp_inven_day_rslt = [x.split(":") for x in inven_d[1].values.tolist()[0].split('/')]
tmp_inven_day_rslt = pd.DataFrame(tmp_inven_day_rslt)
tmp_inven_day_rslt.columns = ['ww', 'inven', 'inven_pred', 'sale', 'day']
inven_day_rslt = tmp_inven_day_rslt[['inven', 'inven_pred', 'sale', 'day']]
inven_day_rslt = inven_day_rslt.astype('float')
inven_day_rslt['ww'] = [x[:6] for x in tmp_inven_day_rslt['ww']]
inven_day_rslt['inven'] = inven_day_rslt['inven'].replace(-999, np.nan)
tmp_s = inven_day_rslt.index[inven_day_rslt['ww'] == '202101'][0]
tmp_e = inven_day_rslt.shape[0]

In [5]:
inven_merge = pd.merge(inven_day_rslt, inven, how='left', on='ww')#[inven['ww'] >= 202101]
col1 = np.where(inven_merge.columns.values == 'inven')[0][0]
col2 = np.where(inven_merge.columns.values == 'inven_true')[0][0]
inven_merge.iloc[tmp_s-1, col2] =inven_merge.iloc[tmp_s-1, col1]
inven_merge1 = pd.merge(inven_merge, lt_d, how='left', on='ww')

In [10]:
inven_merge1['drive_day'] = [2] * inven_merge1.shape[0]

In [11]:
inven_merge1

,inven,inven_pred,sale,day,ww,inven_true,lt_d,drive_day
0,595718.0,626556.809799,120960.0,0.0,202003,NaN,0.4,2
1,561899.0,601158.820831,120960.0,42.7,202004,NaN,0.8,2
2,592359.0,595071.541687,105840.0,37.3,202005,NaN,0.3,2
3,626276.0,607479.543640,74844.0,38.0,202006,NaN,0.0,2
4,537030.0,544863.231537,75600.0,36.3,202007,NaN,0.0,2
...,...,...,...,...,...,...,...,...
96,NaN,869141.410469,325080.0,23.6,202146,NaN,0.0,2
97,NaN,820963.256335,325080.0,26.0,202147,NaN,0.0,2
98,NaN,778585.075223,192628.0,0.0,202148,NaN,0.0,2
99,NaN,890981.081762,192620.0,0.0,202149,NaN,0.0,2


In [14]:
inven_merge.dtypes
#lt_d.dtypes

inven         float64
inven_pred    float64
sale          float64
day           float64
ww             object
inven_true    float64
dtype: object

In [85]:
inven_day_rslt['ww'].loc[52] == inven['ww'].loc[1]

False

In [86]:
inven['ww'].loc[1]

202102

In [88]:
inven_day_rslt.dtypes

inven         float64
inven_pred    float64
sale          float64
day           float64
ww             object
dtype: object